In [ ]:
"""
MARKETING ANALYTICS AI AGENT
=============================
An AI-powered assistant that helps analyze marketing campaign performance
using Claude's API with custom tools for metric calculation and data retrieval.

Author: Aryan reddy loka
Date: November 2024
Technologies: Python, Anthropic Claude API
"""


import anthropic

# Setup - PUT YOUR API KEY HERE
client = anthropic.Anthropic(api_key="YOUR API KEY")

# NOTE: Never commit your API key to version control!
# Consider using environment variables in production:
# client = anthropic.Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))

In [ ]:
# ============================================
# TOOL DEFINITIONS
# ============================================
# These tools define what actions the agent can take.
# Each tool has a name, description, and input schema
# that Claude uses to decide when and how to use them.

# Define tools
tools = [
            {
            "name":"calculate_ctr",
            "description":"Calculate click-through rate (CTR) percentage given clicks and impressions",
            "input_schema":{
                "type":"object",
                "properties":{
                    "clicks":{
                    "type":"number",
                    "description":"number of clicks"
                    },
                    "impressions":{
                        "type":"number",
                        "description":" number of impressions"
                    }
                },
                "required":["clicks","impressions"]
            }
        },
        {
            "name":"calculate_conversion_rate",
            "description":"Calculate conversion rate percentage given conversions and total visitors",
            "input_schema":{
                "type":"object",
                "properties":{
                    "conversions":{
                        "type":"number",
                        "description":" number of conversions"
                    },
                    "visitors":{
                        "type":"number",
                        "description":"total number of visitors"
                    }
                },
                "required":["conversions","visitors"]
            }
        },
        {
            "name":"calculate_roi",
            "description":"Calculate return on investment (ROI) percentage given revenue and cost",
            "input_schema":{
                "type":"object",
                "properties":{
                    "revenue": {
                        "type":"number",
                        "description": "Revenue generated"
                    },
                    "cost":{
                        "type":"number",
                        "description":"Cost of investment"
                    }
                },
                "required":["revenue","cost"]
            }
        },
        {
        "name": "get_campaign_data",
        "description": "Retrieve campaign performance data from the database by campaign name",
        "input_schema": {
            "type": "object",
            "properties": {
                "campaign_name": {
                    "type": "string",
                    "description": "The name of the campaign to retrieve data for"
                }
            },
            "required": ["campaign_name"]
            }
        }
    
]



In [ ]:
# Tool function
#ctr calculator
def calculate_ctr(clicks, impressions):
    if impressions == 0:
        return "Error: Cannot calculate CTR with 0 impressions"
    ctr = (clicks / impressions) * 100
    return f"CTR: {ctr:.2f}%"
#conversion calculator
def calculate_conversion_rate(visitors, conversions):
    if visitors==0:
        return "Error: cannot calculate conversion rate with 0 visitors"
    conversion_rate= (conversions/visitors)*100
    return f"Conversion Rate: {conversion_rate:.2f}%"

#calculate ROI tool
def calculate_roi(revenue, cost):
    if cost ==0:
        return "Error: cannot calculate ROI with 0 cost"
    roi = ((revenue - cost)/cost)*100
    return f"ROI: {roi:.2f}%"
def get_campaign_data(campaign_name):
    """
    Simulate fetching campaign data from a database.
    In a real application, this would connect to your actual database.
    """
    # Simulated database
    campaigns_db = {
        "summer_sale": {
            "campaign_name": "Summer Sale 2024",
            "impressions": 50000,
            "clicks": 1200,
            "conversions": 85,
            "cost": 3500,
            "revenue": 12000,
            "start_date": "2024-06-01",
            "end_date": "2024-08-31"
        },
        "winter_promo": {
            "campaign_name": "Winter Promotion",
            "impressions": 30000,
            "clicks": 900,
            "conversions": 60,
            "cost": 2000,
            "revenue": 8000,
            "start_date": "2024-12-01",
            "end_date": "2024-12-31"
        },
        "spring_launch": {
            "campaign_name": "Spring Product Launch",
            "impressions": 75000,
            "clicks": 2250,
            "conversions": 150,
            "cost": 5000,
            "revenue": 18000,
            "start_date": "2024-03-01",
            "end_date": "2024-05-31"
        }
    }
    
    # Look up campaign
    campaign = campaigns_db.get(campaign_name.lower().replace(" ", "_"))
    
    if campaign:
        # Format the data nicely
        return f"""Campaign Data Retrieved:
        
            Name: {campaign['campaign_name']}
            Period: {campaign['start_date']} to {campaign['end_date']}
            Impressions: {campaign['impressions']:,}
            Clicks: {campaign['clicks']:,}
            Conversions: {campaign['conversions']}
            Cost: ${campaign['cost']:,}
            Revenue: ${campaign['revenue']:,}"""
    else:
        return f"Campaign '{campaign_name}' not found in database. Available campaigns: summer_sale, winter_promo, spring_launch"
#Execute tool based on name
def execute_tool(tool_name, tool_input):
    if tool_name =='calculate_ctr':
        return calculate_ctr(tool_input['clicks'], tool_input["impressions"])
    elif tool_name =='calculate_conversion_rate':
        return calculate_conversion_rate(tool_input["visitors"],tool_input["conversions"])
    elif tool_name =='calculate_roi':
        return calculate_roi(tool_input['revenue'], tool_input['cost'])
    elif tool_name == "get_campaign_data":
        return get_campaign_data(tool_input["campaign_name"])
    
    return "Unknown tool"

# Main agent function
def run_agent(user_question):
    """Run the marketing analytics agent"""
    print(f"\n{'='*60}")
    print(f"USER: {user_question}")
    print(f"{'='*60}\n")
    
    messages = [{"role": "user", "content": user_question}]
    
    # Keep looping while Claude needs tools
    while True:
        response = client.messages.create(
            model="claude-sonnet-4-5",
            max_tokens=1024,
            tools=tools,
            messages=messages
        )
        
        # Check if Claude is done
        if response.stop_reason == "end_turn":
            # Extract text response
            for block in response.content:
                if hasattr(block, "text"):
                    print(f"CLAUDE: {block.text}\n")
            break
        
        # If Claude wants to use tools
        if response.stop_reason == "tool_use":
            # Add Claude's response to messages
            messages.append({"role": "assistant", "content": response.content})
            
            # Collect ALL tool results
            tool_results = []
            
            # Process each tool use in the response
            for block in response.content:
                if block.type == "tool_use":
                    tool_name = block.name
                    tool_input = block.input
                    tool_id = block.id
                    
                    print(f"🔧 Using: {tool_name}")
                    print(f"   Input: {tool_input}\n")
                    
                    # Execute tool
                    result = execute_tool(tool_name, tool_input)
                    print(f"✅ Result: {result}\n")
                    
                    # Add to results
                    tool_results.append({
                        "type": "tool_result",
                        "tool_use_id": tool_id,
                        "content": result
                    })
            
            # Send ALL tool results back at once
            messages.append({
                "role": "user",
                "content": tool_results
            })
            
            # Loop continues - Claude will process all results
# Run it!
if __name__ == "__main__":
    print("\n🎯 MARKETING ANALYTICS AGENT")
    print("Ask me to calculate marketing metrics or retrieve campaign data!\n")
    
    # Test basic calculations
    run_agent("What's the CTR for 200 clicks and 10000 impressions?")
    run_agent("Calculate ROI if revenue was $25000 and cost was $8000")
    
    # Test data retrieval
    run_agent("Show me the data for the summer_sale campaign")
    
    # Test combined workflow
    run_agent("Get the winter_promo campaign data and calculate its ROI")
    run_agent("Compare the ROI of summer_sale and winter_promo campaigns")